In [1]:

#Importar librerias? Pues claro que me importan!
import numpy as np
from numpy import random
from random import choices 
import os 


In [2]:

def asintomatico_o_no():
    estado = ["asintomatico", "sintomatico"]
    peso_de_decision = [0.6, 0.4]
    n = choices(estado, peso_de_decision)
    return n[0]

def probabilidad_de_muerte(edad):
    if(edad > 65):
        valoracion = [0,1]
        peso = [0.87, 0.13] #queda pendiente revisar esto 
        n = choices(valoracion, peso)
    elif(edad <= 65):
        valoracion = [0,1]
        peso = [0.99, 0.01] #probabilidad que se correspende en caso que no tenga antecentes de ningun tipo
        n = choices(valoracion, peso) #n es un vector cuyo primer elemento es la valoracion 
    return n[0]
    

In [3]:
#Objeto que simula a una persona.
class Persona:
        def __init__(self, idpersona,edad,lugaroriginal, dni):           
            self.idpersona=idpersona
            self.edad=edad
            self.estadosposibles=["sano","incubando","incubandoContagioso","sintomatico","asintomatico","muerto","inmune"]
            self.estado=self.estadosposibles[0]
            self.lugar=lugaroriginal
            self.diasestado=0
            self.cambioEstado=np.inf  #Atributo que indica que dia cambiara la persona de estado.
            self.contadorInfecciones=0
            self.dni = dni
            self.edificiomuerte=-1
            
            
           
        def __str__(self):
            #Devuelve un string con la informacion util del sujeto
            return str(self.idpersona) + "," + str(self.estado) + "," + str(self.lugar) + "," +str(self.dni) + "," +str(self.edad)            

        def puede_propagar(self):
            return ((self.estado == "sintomatico") or (self.estado == "asintomatico") or (self.estado == "incubandoContagioso"))
        def susceptible(self):
            return (self.estado == "sano")
        
        def esta_sano(self):
            return (self.estado == "sano") or (self.estado == "inmune")
        
        def infectar(self,posibilidadContagio,ciudad,mediaincubacion,desvincubacion,diaactual): #media y desv covid: 7.5 y 3.5
            if self.puede_propagar():
                for i in range(len(ciudad[self.lugar])): 
                    if (ciudad[self.lugar][i].susceptible()):
                        nocontagia=random.uniform(0, 1)   #Numero al azar, si es menor que la pos. de contagio, contagiara
                        if (nocontagia <posibilidadContagio/24):
                            ciudad[self.lugar][i].estado=ciudad[self.lugar][i].estadosposibles[1]
                            ciudad[self.lugar][i].diasestado=0
                            ciudad[self.lugar][i].cambioEstado=diaactual+(random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2)
                           # print(ciudad[self.lugar][i].cambioEstado)
                            self.contadorInfecciones +=1
                            
        def transicionEstados(self, dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            #A partir de aqui se comprueba que estado posee la persona
            #Y se mira dos casos:
            #Si el dia del cambio de estado , coincide con el dia actual
            if(self.estado == "sintomatico"):
                me_voy_a_morir = probabilidad_de_muerte(self.edad)
                if(me_voy_a_morir == 1):
                    self.cambioEstado=np.inf
                    self.estado = "muerto"
            
            if(self.cambioEstado <= dia):
                if(self.estado == "incubando"):
                    self.estado = "incubandoContagioso" #se le asigna el siguiente estado  
                    self.cambioEstado = dia + random.randint(int(mediaincubacion-desvincubacion),int(mediaincubacion+desvincubacion))/2    #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "incubandoContagioso"):
                    self.estado = asintomatico_o_no() #se le asigna el siguiente estado 
                    self.cambioEstado =dia+ random.randint(int(mediaduracion-desvduracion),int(mediaduracion+desvduracion)) #tiempo que pasa a inmune o muerto xD

                elif(self.estado == "sintomatico"):
                    self.cambioEstado = np.inf 
                    self.estado = "inmune"
                    '''
                      Y ahora una vez que pasa a ser asintomatico o no, se debe ver la probabilidad de que la persona se vaya 
                       a morir o no, y dependiendo de eso, se ha asignan unos dias...
                    '''

                elif(self.estado == "asintomatico"):
                    self.estado = "inmune"
                    self.cambioEstado = np.inf 
                                     
                

In [200]:

class Simulador:
        def __init__(self, numpersonas,personasedificio):           
            self.ciudad=crearciudadv0(numpersonas,personasedificio)
            self.dia=0 #se inicializa el dia con un valor de 0 
            self.hora=0 #se inicializa las horas con un valor de 0 
            self.numpersonas=numpersonas 
            self.numedificios=len(self.ciudad)
            self.capacidades=np.zeros(self.numedificios)
            self.CatalogoPersonas=[]
            for i in range(self.numedificios):
                self.capacidades[i]=int(len(self.ciudad[i])) #establecemos las capacidades de los edificios
                for j in range (len(self.ciudad[i])):
                    self.CatalogoPersonas.append(self.ciudad[i][j])
                    
            self.capacidades=self.capacidades.astype(int)
            print(self.capacidades)
            for i in range(len(self.CatalogoPersonas)):
                print(self.CatalogoPersonas[i])
            
                
           
        def __str__(self):
            #Devuelve un string con la informacion util del simulador
            return str(self.dia) + "," + str(self.hora) + "," + str(self.numpersonas) + "," +str(self.numedificios)  
        def actualizar_catalogo(self):#actualizamos los datos de las personas ordenadas por id
            nuevoCatalogo=[]
            for i in range(self.numedificios):
                for j in range (len(self.ciudad[i])):
                    nuevoCatalogo.append(self.ciudad[i][j]) #primero cogemos todas las personas sin ordenar
                    self.CatalogoPersonas = sorted(nuevoCatalogo, key=lambda x: x.idpersona, reverse=False)
            
        #Hace que el simulador avance una hora , con sus consecuencias en las personas
        def pasar_tiempo(self,mediaincubacion,desvincubacion,mediaduracion,desvduracion):
            if ((self.hora % 8)==0): #barajar a las personas cada 8 horas( hacen shuffle)
                self.moverpersonas()
                print("owo")
            for j in range(len(self.ciudad)): #recorre los edificios 
                for i in range(len(self.ciudad[j])): #recorre las personas 
                    self.ciudad[j][i].infectar(0.4,self.ciudad,mediaincubacion,desvincubacion,self.dia) #pasa a infectar 
            if (self.hora<23):
                self.hora+=1
            else:
                self.hora=0
                self.dia+=1
                for j in range(len(self.ciudad)):
                    for i in range(len(self.ciudad[j])):
                        self.ciudad[j][i].transicionEstados(self.dia, mediaincubacion,desvincubacion,mediaduracion,desvduracion)
        
        #Función que devuelve el total de personas sanas o que ya pasaron la enfermedad de la ciudad
        def personas_sanas(self):
            contador_sanos=0
            for j in range(len(self.ciudad)): #se recorre los edificios de la ciudad
                    for i in range(len(self.ciudad[j])): #se va recorriendo las personas de los edificios 
                        if self.ciudad[j][i].esta_sano():
                            contador_sanos+=1
            return contador_sanos
        
        def cogerpersona(self, idpersona):
            self.actualizar_catalogo()
            return self.CatalogoPersonas[idpersona]
        
        def moverpersonas(self):#Baraja a todas las personas de la ciudad indiscriminadamente
                                #Vamos a crear una nueva ciudad utilizando las personas actuales
            edificiosSinVaciar=self.numedificios
            ciudadnueva=[] #la nueva ciudad
            for i in range(self.numedificios):
                edificionuevo = [] #El edificio actual de la ciudad nueva.
                for j in range(self.capacidades[i]):
                    if edificiosSinVaciar>1:
                        edificioescogido=random.randint(0,edificiosSinVaciar-1)
                    else:
                        edificioescogido=0
                    edificionuevo.append(self.ciudad[edificioescogido].pop(0))
                    if (len(self.ciudad[edificioescogido])==0):
                        del  self.ciudad[edificioescogido]
                        edificiosSinVaciar-=1
            
                ciudadnueva.append(edificionuevo)
            self.ciudad=ciudadnueva
        #Función que contagia a la primera persona (la deja sintomática), se usa para crear pacientes 0
        def contagio_fijo(self):
            self.ciudad[0][0].estado=self.ciudad[0][0].estadosposibles[3]
            self.ciudad[0][0].cambioEstado=15

In [201]:
#Funcion que se encarga de generar la edad de manera aleatoria
def generador_edad():
        n = random.randint(0,110) #el rango de edad, sera desde los 0 años hasta los 110
        return n


In [202]:
#Funcion que se encarga de generar numeros aleatorios 
def generador_numeros(longitud):
    numero=0
    for i in range(0,longitud): 
        n = random.randint(0,9)
        numero=10*numero+n
    return numero


In [203]:
#Funcion que se encarga de crear una secuencia valida de DNI
#Para ello, se ha utilizado el algoritmo que se usa en la vida real -en cuanto a la relacion numero y letra- 
def creaDni():
    letras = "TRWAGMYFPDXBNJZSQVHLCKEO" #conjunto de letras del dni
    input=generador_numeros(8)
    #Se consigue que toda la secuencia de numeros sea 1 solo
    
    #Se calcula la letra que ha de ser adicionada
    valor =int(input / 23)
    #print(f"El valor en este caso es: {valor}")
    valor *= 23
    valor = input - valor;
    #Ahora, ha de pasarse el numero de la secuencia
    #a una lista donde cada elemento es un numero de la secuencia 
    DNI=str(input)+letras[valor]
    
    return DNI

In [204]:
#Esta función crea la ciudad en la versión 0 de la simulacion.
#La ciudad es una lista de edificios y cada edificio es una lista de personas de tamaño variable.
#Input: personas de la ciudad, media de personas por edificio.
#Output: la ciudad.
def crearciudadv0(numpersonas,personasedificio):
    edificios = [] #La ciudad.
    capacidadusada=0 #Numero de personas metidas en la ciudad.
    edificioactual=0
    while (capacidadusada < numpersonas):
        column = [] #El edificio.
        esteedificio=random.randint(1,personasedificio*3) 
        '''
        Maximo de personas en el edificio. 
        Si usamos desviación tipica de capacidad de edificio, iria aqui
        '''
        contadoredificio=0 #numero de personas colocadas en este edificio
        #print(f"edificio actual:{edificioactual}")
        
        #Bucle en el que se llena el edificio de personas.
        while (capacidadusada < numpersonas) and (contadoredificio < esteedificio):
            
            #print(f"contador del edificio:{contadoredificio}")
            #print(f"capacidad:{esteedificio}")
            
            #Insercion de persona.
            column.append(Persona(capacidadusada,generador_edad(),edificioactual, creaDni()))
            capacidadusada+=1
            contadoredificio+=1
            
            
        #Insercion de edificio en la ciudad
        edificios.append(column)
        edificioactual+=1
        
    return edificios
def printearciudad(ciudad):        
    for j in range(len(ciudad)):
        for i in range(len(ciudad[j])):
            print(ciudad[j][i])
        print()        
        
#Funcion que dada la ciudad, y un indice, permite imprimir el edificio que corresponda a ese indice
def printearEdificio(ciudad , i):
    for j in range(len(ciudad[i])):
        print(ciudad[i][j])
    

In [214]:
MiSimulador=Simulador(200,200) #numPersonas, personasEdificio?? 
print(MiSimulador)
MiSimulador.contagio_fijo() #se crea un paciente que resulta ser el 0 
for i in range(700):
    '''
    Datos inventados:
    MediaIncubación:7 dias
    DesvIncubación:3 dias
    MediaDuracion:14 dias
    DescDuracion:5 dias
    '''
    MiSimulador.pasar_tiempo(7,3,14,5)
    if MiSimulador.hora==0:
        print("--------")
        print(" ")
        print(" ")
        print(MiSimulador.dia)
        print(MiSimulador)
        printearciudad(MiSimulador.ciudad)
        print(MiSimulador.personas_sanas())
        print(" ")
        print(" ")
        print("--------")

[200]
0,sano,0,36347640G,91
1,sano,0,58242247Z,15
2,sano,0,71512140B,12
3,sano,0,73211817B,80
4,sano,0,86877383N,67
5,sano,0,23580287C,3
6,sano,0,18284634W,15
7,sano,0,8724774T,14
8,sano,0,50002872K,108
9,sano,0,48621707Y,87
10,sano,0,47448121H,6
11,sano,0,88545446T,23
12,sano,0,10256744D,83
13,sano,0,10087788B,15
14,sano,0,14354155Q,42
15,sano,0,52532122F,60
16,sano,0,36402626C,56
17,sano,0,87807587G,66
18,sano,0,44457253R,13
19,sano,0,31246866D,7
20,sano,0,18440182R,61
21,sano,0,11164754W,15
22,sano,0,62324480T,100
23,sano,0,43227352W,93
24,sano,0,73614170W,5
25,sano,0,42287021A,40
26,sano,0,74551214A,47
27,sano,0,75473128P,16
28,sano,0,48670047T,106
29,sano,0,57244877Q,93
30,sano,0,55576131N,21
31,sano,0,22228127F,53
32,sano,0,4026161B,28
33,sano,0,1843657T,23
34,sano,0,58068205J,71
35,sano,0,41487136N,83
36,sano,0,28185884M,35
37,sano,0,44784732Y,26
38,sano,0,26687880Z,66
39,sano,0,34248646J,32
40,sano,0,56757580C,34
41,sano,0,78515485V,60
42,sano,0,72537183J,26
43,sano,0,14481176P

owo
--------
 
 
4
4,0,200,1
0,sintomatico,0,36347640G,91
1,incubandoContagioso,0,58242247Z,15
2,incubando,0,71512140B,12
3,incubando,0,73211817B,80
4,incubando,0,86877383N,67
5,incubando,0,23580287C,3
6,incubandoContagioso,0,18284634W,15
7,incubando,0,8724774T,14
8,incubandoContagioso,0,50002872K,108
9,incubandoContagioso,0,48621707Y,87
10,incubando,0,47448121H,6
11,incubando,0,88545446T,23
12,incubando,0,10256744D,83
13,incubandoContagioso,0,10087788B,15
14,incubando,0,14354155Q,42
15,incubando,0,52532122F,60
16,incubandoContagioso,0,36402626C,56
17,incubando,0,87807587G,66
18,incubando,0,44457253R,13
19,incubandoContagioso,0,31246866D,7
20,incubando,0,18440182R,61
21,incubando,0,11164754W,15
22,incubandoContagioso,0,62324480T,100
23,incubando,0,43227352W,93
24,incubando,0,73614170W,5
25,incubandoContagioso,0,42287021A,40
26,incubando,0,74551214A,47
27,incubando,0,75473128P,16
28,incubando,0,48670047T,106
29,incubandoContagioso,0,57244877Q,93
30,incubandoContagioso,0,55576131N,21
31,

--------
 
 
6
6,0,200,1
0,sintomatico,0,36347640G,91
1,incubandoContagioso,0,58242247Z,15
2,incubando,0,71512140B,12
3,incubandoContagioso,0,73211817B,80
4,incubandoContagioso,0,86877383N,67
5,incubandoContagioso,0,23580287C,3
6,incubandoContagioso,0,18284634W,15
7,incubandoContagioso,0,8724774T,14
8,incubandoContagioso,0,50002872K,108
9,incubandoContagioso,0,48621707Y,87
10,incubandoContagioso,0,47448121H,6
11,incubandoContagioso,0,88545446T,23
12,incubandoContagioso,0,10256744D,83
13,incubandoContagioso,0,10087788B,15
14,incubandoContagioso,0,14354155Q,42
15,incubandoContagioso,0,52532122F,60
16,incubandoContagioso,0,36402626C,56
17,incubandoContagioso,0,87807587G,66
18,incubandoContagioso,0,44457253R,13
19,incubandoContagioso,0,31246866D,7
20,incubando,0,18440182R,61
21,incubandoContagioso,0,11164754W,15
22,asintomatico,0,62324480T,100
23,incubandoContagioso,0,43227352W,93
24,incubandoContagioso,0,73614170W,5
25,incubandoContagioso,0,42287021A,40
26,incubandoContagioso,0,74551214A,

--------
 
 
8
8,0,200,1
0,muerto,0,36347640G,91
1,incubandoContagioso,0,58242247Z,15
2,incubandoContagioso,0,71512140B,12
3,incubandoContagioso,0,73211817B,80
4,asintomatico,0,86877383N,67
5,asintomatico,0,23580287C,3
6,sintomatico,0,18284634W,15
7,incubandoContagioso,0,8724774T,14
8,asintomatico,0,50002872K,108
9,asintomatico,0,48621707Y,87
10,incubandoContagioso,0,47448121H,6
11,incubandoContagioso,0,88545446T,23
12,incubandoContagioso,0,10256744D,83
13,asintomatico,0,10087788B,15
14,incubandoContagioso,0,14354155Q,42
15,incubandoContagioso,0,52532122F,60
16,asintomatico,0,36402626C,56
17,incubandoContagioso,0,87807587G,66
18,sintomatico,0,44457253R,13
19,asintomatico,0,31246866D,7
20,incubandoContagioso,0,18440182R,61
21,incubandoContagioso,0,11164754W,15
22,asintomatico,0,62324480T,100
23,asintomatico,0,43227352W,93
24,incubandoContagioso,0,73614170W,5
25,sintomatico,0,42287021A,40
26,incubandoContagioso,0,74551214A,47
27,incubandoContagioso,0,75473128P,16
28,incubandoContagioso,0

--------
 
 
10
10,0,200,1
0,muerto,0,36347640G,91
1,sintomatico,0,58242247Z,15
2,asintomatico,0,71512140B,12
3,sintomatico,0,73211817B,80
4,asintomatico,0,86877383N,67
5,asintomatico,0,23580287C,3
6,sintomatico,0,18284634W,15
7,sintomatico,0,8724774T,14
8,asintomatico,0,50002872K,108
9,asintomatico,0,48621707Y,87
10,sintomatico,0,47448121H,6
11,asintomatico,0,88545446T,23
12,asintomatico,0,10256744D,83
13,asintomatico,0,10087788B,15
14,sintomatico,0,14354155Q,42
15,sintomatico,0,52532122F,60
16,asintomatico,0,36402626C,56
17,asintomatico,0,87807587G,66
18,sintomatico,0,44457253R,13
19,asintomatico,0,31246866D,7
20,incubandoContagioso,0,18440182R,61
21,sintomatico,0,11164754W,15
22,asintomatico,0,62324480T,100
23,asintomatico,0,43227352W,93
24,incubandoContagioso,0,73614170W,5
25,sintomatico,0,42287021A,40
26,asintomatico,0,74551214A,47
27,incubandoContagioso,0,75473128P,16
28,asintomatico,0,48670047T,106
29,asintomatico,0,57244877Q,93
30,sintomatico,0,55576131N,21
31,asintomatico,0,22

...

--------
 
 
16
16,0,200,1
0,muerto,0,36347640G,91
1,sintomatico,0,58242247Z,15
2,asintomatico,0,71512140B,12
3,muerto,0,73211817B,80
4,asintomatico,0,86877383N,67
5,asintomatico,0,23580287C,3
6,inmune,0,18284634W,15
7,sintomatico,0,8724774T,14
8,asintomatico,0,50002872K,108
9,asintomatico,0,48621707Y,87
10,sintomatico,0,47448121H,6
11,asintomatico,0,88545446T,23
12,asintomatico,0,10256744D,83
13,asintomatico,0,10087788B,15
14,sintomatico,0,14354155Q,42
15,sintomatico,0,52532122F,60
16,asintomatico,0,36402626C,56
17,asintomatico,0,87807587G,66
18,sintomatico,0,44457253R,13
19,inmune,0,31246866D,7
20,sintomatico,0,18440182R,61
21,sintomatico,0,11164754W,15
22,asintomatico,0,62324480T,100
23,asintomatico,0,43227352W,93
24,sintomatico,0,73614170W,5
25,sintomatico,0,42287021A,40
26,asintomatico,0,74551214A,47
27,asintomatico,0,75473128P,16
28,asintomatico,0,48670047T,106
29,asintomatico,0,57244877Q,93
30,muerto,0,55576131N,21
31,asintomatico,0,22228127F,53
32,asintomatico,0,4026161B,28
33,

6,inmune,0,18284634W,15
7,sintomatico,0,8724774T,14
8,asintomatico,0,50002872K,108
9,asintomatico,0,48621707Y,87
10,sintomatico,0,47448121H,6
11,asintomatico,0,88545446T,23
12,asintomatico,0,10256744D,83
13,inmune,0,10087788B,15
14,sintomatico,0,14354155Q,42
15,sintomatico,0,52532122F,60
16,asintomatico,0,36402626C,56
17,asintomatico,0,87807587G,66
18,inmune,0,44457253R,13
19,inmune,0,31246866D,7
20,sintomatico,0,18440182R,61
21,sintomatico,0,11164754W,15
22,asintomatico,0,62324480T,100
23,asintomatico,0,43227352W,93
24,sintomatico,0,73614170W,5
25,sintomatico,0,42287021A,40
26,asintomatico,0,74551214A,47
27,asintomatico,0,75473128P,16
28,asintomatico,0,48670047T,106
29,asintomatico,0,57244877Q,93
30,muerto,0,55576131N,21
31,asintomatico,0,22228127F,53
32,inmune,0,4026161B,28
33,muerto,0,1843657T,23
34,muerto,0,58068205J,71
35,asintomatico,0,41487136N,83
36,sintomatico,0,28185884M,35
37,asintomatico,0,44784732Y,26
38,inmune,0,26687880Z,66
39,sintomatico,0,34248646J,32
40,sintomatico,0,

owo
--------
 
 
21
21,0,200,1
0,muerto,0,36347640G,91
1,inmune,0,58242247Z,15
2,asintomatico,0,71512140B,12
3,muerto,0,73211817B,80
4,asintomatico,0,86877383N,67
5,asintomatico,0,23580287C,3
6,inmune,0,18284634W,15
7,inmune,0,8724774T,14
8,inmune,0,50002872K,108
9,inmune,0,48621707Y,87
10,sintomatico,0,47448121H,6
11,asintomatico,0,88545446T,23
12,inmune,0,10256744D,83
13,inmune,0,10087788B,15
14,sintomatico,0,14354155Q,42
15,inmune,0,52532122F,60
16,inmune,0,36402626C,56
17,asintomatico,0,87807587G,66
18,inmune,0,44457253R,13
19,inmune,0,31246866D,7
20,sintomatico,0,18440182R,61
21,sintomatico,0,11164754W,15
22,inmune,0,62324480T,100
23,asintomatico,0,43227352W,93
24,sintomatico,0,73614170W,5
25,inmune,0,42287021A,40
26,inmune,0,74551214A,47
27,asintomatico,0,75473128P,16
28,asintomatico,0,48670047T,106
29,asintomatico,0,57244877Q,93
30,muerto,0,55576131N,21
31,inmune,0,22228127F,53
32,inmune,0,4026161B,28
33,muerto,0,1843657T,23
34,muerto,0,58068205J,71
35,inmune,0,41487136N,83
36,s

owo
--------
 
 
23
23,0,200,1
0,muerto,0,36347640G,91
1,inmune,0,58242247Z,15
2,asintomatico,0,71512140B,12
3,muerto,0,73211817B,80
4,inmune,0,86877383N,67
5,asintomatico,0,23580287C,3
6,inmune,0,18284634W,15
7,inmune,0,8724774T,14
8,inmune,0,50002872K,108
9,inmune,0,48621707Y,87
10,sintomatico,0,47448121H,6
11,asintomatico,0,88545446T,23
12,inmune,0,10256744D,83
13,inmune,0,10087788B,15
14,sintomatico,0,14354155Q,42
15,inmune,0,52532122F,60
16,inmune,0,36402626C,56
17,inmune,0,87807587G,66
18,inmune,0,44457253R,13
19,inmune,0,31246866D,7
20,sintomatico,0,18440182R,61
21,inmune,0,11164754W,15
22,inmune,0,62324480T,100
23,inmune,0,43227352W,93
24,inmune,0,73614170W,5
25,inmune,0,42287021A,40
26,inmune,0,74551214A,47
27,inmune,0,75473128P,16
28,asintomatico,0,48670047T,106
29,asintomatico,0,57244877Q,93
30,muerto,0,55576131N,21
31,inmune,0,22228127F,53
32,inmune,0,4026161B,28
33,muerto,0,1843657T,23
34,muerto,0,58068205J,71
35,inmune,0,41487136N,83
36,sintomatico,0,28185884M,35
37,asint

165,inmune,0,31778037H,32
166,muerto,0,58554107V,91
167,inmune,0,60281760W,103
168,inmune,0,73884887D,56
169,inmune,0,58202142K,16
170,inmune,0,46801505R,1
171,inmune,0,62006447B,60
172,inmune,0,70818737N,44
173,asintomatico,0,20068145R,17
174,inmune,0,64572465B,33
175,inmune,0,48235638S,36
176,inmune,0,72218281Y,63
177,inmune,0,22860122P,21
178,inmune,0,53327347F,67
179,inmune,0,8855388C,95
180,muerto,0,81443725N,12
181,inmune,0,72133756Y,22
182,inmune,0,7036064L,80
183,inmune,0,70604418F,37
184,inmune,0,60727805P,94
185,inmune,0,621566Z,8
186,inmune,0,18855414Z,70
187,inmune,0,3376030K,56
188,sintomatico,0,30270706S,59
189,inmune,0,64650674C,70
190,inmune,0,37586644K,80
191,inmune,0,61068687F,109
192,inmune,0,62428830E,88
193,inmune,0,81021055J,102
194,inmune,0,88256848Y,21
195,sintomatico,0,53224647W,26
196,inmune,0,20583305F,93
197,inmune,0,57086327M,27
198,inmune,0,47186218Q,44
199,inmune,0,48542286G,3

157
 
 
--------
owo
owo
owo
--------
 
 
27
27,0,200,1
0,muerto,0,36347640G,9

In [215]:
printearEdificio(MiSimulador.ciudad,0)

0,muerto,0,36347640G,91
1,inmune,0,58242247Z,15
2,inmune,0,71512140B,12
3,muerto,0,73211817B,80
4,inmune,0,86877383N,67
5,inmune,0,23580287C,3
6,inmune,0,18284634W,15
7,inmune,0,8724774T,14
8,inmune,0,50002872K,108
9,inmune,0,48621707Y,87
10,inmune,0,47448121H,6
11,inmune,0,88545446T,23
12,inmune,0,10256744D,83
13,inmune,0,10087788B,15
14,inmune,0,14354155Q,42
15,inmune,0,52532122F,60
16,inmune,0,36402626C,56
17,inmune,0,87807587G,66
18,inmune,0,44457253R,13
19,inmune,0,31246866D,7
20,inmune,0,18440182R,61
21,inmune,0,11164754W,15
22,inmune,0,62324480T,100
23,inmune,0,43227352W,93
24,inmune,0,73614170W,5
25,inmune,0,42287021A,40
26,inmune,0,74551214A,47
27,inmune,0,75473128P,16
28,inmune,0,48670047T,106
29,inmune,0,57244877Q,93
30,muerto,0,55576131N,21
31,inmune,0,22228127F,53
32,inmune,0,4026161B,28
33,muerto,0,1843657T,23
34,muerto,0,58068205J,71
35,inmune,0,41487136N,83
36,inmune,0,28185884M,35
37,inmune,0,44784732Y,26
38,inmune,0,26687880Z,66
39,inmune,0,34248646J,32
40,inmune,0,56

In [216]:
printearciudad(MiSimulador.ciudad)

0,muerto,0,36347640G,91
1,inmune,0,58242247Z,15
2,inmune,0,71512140B,12
3,muerto,0,73211817B,80
4,inmune,0,86877383N,67
5,inmune,0,23580287C,3
6,inmune,0,18284634W,15
7,inmune,0,8724774T,14
8,inmune,0,50002872K,108
9,inmune,0,48621707Y,87
10,inmune,0,47448121H,6
11,inmune,0,88545446T,23
12,inmune,0,10256744D,83
13,inmune,0,10087788B,15
14,inmune,0,14354155Q,42
15,inmune,0,52532122F,60
16,inmune,0,36402626C,56
17,inmune,0,87807587G,66
18,inmune,0,44457253R,13
19,inmune,0,31246866D,7
20,inmune,0,18440182R,61
21,inmune,0,11164754W,15
22,inmune,0,62324480T,100
23,inmune,0,43227352W,93
24,inmune,0,73614170W,5
25,inmune,0,42287021A,40
26,inmune,0,74551214A,47
27,inmune,0,75473128P,16
28,inmune,0,48670047T,106
29,inmune,0,57244877Q,93
30,muerto,0,55576131N,21
31,inmune,0,22228127F,53
32,inmune,0,4026161B,28
33,muerto,0,1843657T,23
34,muerto,0,58068205J,71
35,inmune,0,41487136N,83
36,inmune,0,28185884M,35
37,inmune,0,44784732Y,26
38,inmune,0,26687880Z,66
39,inmune,0,34248646J,32
40,inmune,0,56

In [217]:
MiSimulador.actualizar_catalogo()
for i in range(len(MiSimulador.CatalogoPersonas)):
                print(MiSimulador.CatalogoPersonas[i])
            

0,muerto,0,36347640G,91
1,inmune,0,58242247Z,15
2,inmune,0,71512140B,12
3,muerto,0,73211817B,80
4,inmune,0,86877383N,67
5,inmune,0,23580287C,3
6,inmune,0,18284634W,15
7,inmune,0,8724774T,14
8,inmune,0,50002872K,108
9,inmune,0,48621707Y,87
10,inmune,0,47448121H,6
11,inmune,0,88545446T,23
12,inmune,0,10256744D,83
13,inmune,0,10087788B,15
14,inmune,0,14354155Q,42
15,inmune,0,52532122F,60
16,inmune,0,36402626C,56
17,inmune,0,87807587G,66
18,inmune,0,44457253R,13
19,inmune,0,31246866D,7
20,inmune,0,18440182R,61
21,inmune,0,11164754W,15
22,inmune,0,62324480T,100
23,inmune,0,43227352W,93
24,inmune,0,73614170W,5
25,inmune,0,42287021A,40
26,inmune,0,74551214A,47
27,inmune,0,75473128P,16
28,inmune,0,48670047T,106
29,inmune,0,57244877Q,93
30,muerto,0,55576131N,21
31,inmune,0,22228127F,53
32,inmune,0,4026161B,28
33,muerto,0,1843657T,23
34,muerto,0,58068205J,71
35,inmune,0,41487136N,83
36,inmune,0,28185884M,35
37,inmune,0,44784732Y,26
38,inmune,0,26687880Z,66
39,inmune,0,34248646J,32
40,inmune,0,56

In [218]:
personitaputa=MiSimulador.cogerpersona(0)
print(personitaputa.contadorInfecciones)

123
